# Link to the Notebook

[here]:(https://eu-gb.dataplatform.cloud.ibm.com/analytics/notebooks/v2/13678556-466e-46b9-bb84-ef3cf758c8b9/view?access_token=cd21059b9ce3b80f5400879c1acd97e2e01a3ce54b13d4eff6d40d55d2082993)

In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

In [2]:
neighborhood=pd.DataFrame(df[0])
for i in range(9):
    neighborhood[i]=neighborhood[i].str.replace('([A-Z][\d][A-Z]Not assigned)',"A")

In [3]:
lis=[]
for i in range(9):
    lis+=list(neighborhood[i])
    

In [4]:
tor=pd.DataFrame({'Postal Code':lis})
tor.replace('A',np.nan,inplace=True)
tor.dropna(axis=0,inplace=True)
tor.reset_index(drop=True,inplace=True)

In [5]:
pos=np.array([])
pos=(tor['Postal Code'].str.extract('([A-Z][\d][A-Z])'))
bor=(tor['Postal Code']).str.extract('[A-Z][\d][A-Z]([A-Z][a-z\sA-Z]+[a-z]+)')
nei=tor['Postal Code'].str.extract('\((.+)\)$')

In [6]:
tor.insert(1,'Borough','a')
tor.insert(2,'Neighborhood','a')
tor['Postal Code']=pos
tor['Borough']=bor
tor['Neighborhood']=nei
tor['Neighborhood']=tor['Neighborhood'].str.replace('/',',')
tor.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
tor.shape

(103, 3)

In [8]:
ll=pd.read_csv('http://cocl.us/Geospatial_data')
ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
tor=tor.merge(ll,on='Postal Code')
tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
import re
an=[]
ind=[]
for i in range(tor.shape[0]):
    an.extend(re.findall('([\w\s]+Toronto)',tor.loc[i,'Borough']))
              
an=list(set(an))
an

['Downtown Toronto',
 'East Toronto',
 'West Toronto',
 'Central Toronto',
 'East YorkEast Toronto']

In [12]:
new_df=pd.DataFrame()
for i in an:
    new_df=new_df.append(tor[tor['Borough']==i],ignore_index=True)
new_df.tail()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
32,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049
33,M5N,Central Toronto,Roselawn,43.711695,-79.416936
34,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
35,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678
36,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106


## I am going cluster the Neighborhoods in Toronto

In [13]:
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town , Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [14]:
l=[tor.loc[1,'Latitude'],tor.loc[1,'Longitude']]
l

[43.7845351, -79.16049709999999]

In [15]:
CLIENT_ID='KFLYO0KRSRUTVZMXRYZ1MQHSEHH1GFEANLBCFZCHKE1EK0GN'
CLIENT_SECRET='JONA0JLXBZYB1JCGARW3PJYR4A0UO1NBADVADKVAK5BFGSXL'
VERSION='20180605'
uri= 'https://api.foursqaure.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&radius=500&limit=100&v={}'.format(\
      CLIENT_ID,CLIENT_SECRET,l[0],l[1],VERSION)
lat=l[0]
lng=l[1]
radius=1000
LIMIT=100

In [16]:
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results = requests.get(url).json()
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b96e31cf964a5207deb34e3',
   'name': 'Shamrock Burgers',
   'location': {'address': '6070 Old Kingston Rd.',
    'lat': 43.78382252268771,
    'lng': -79.16840631604676,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.78382252268771,
      'lng': -79.16840631604676}],
    'distance': 640,
    'cc': 'CA',
    'city': 'Scarborough',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['6070 Old Kingston Rd.', 'Scarborough ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16c941735',
     'name': 'Burger Joint',
     'pluralName': 'Burger Joints',
     'shortName': 'Burgers',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4b96e31cf964a5207deb34e3-

## Getting top 100 places for neighborhoods

In [17]:
def get_top_places(names,boro,latitude,longitude):
    radius=1000
    limit=100
    exp=[]
    for name,bor,lat,lng in zip(names,boro,latitude,longitude):
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                limit)
        results = requests.get(url).json()['response']['groups'][0]['items']
        exp.append([[name,bor,lat,lng,v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']] for v in results])
    li=[]
    for e in exp:
        for w in e:
            li.append(w)
    df=pd.DataFrame(li)
    df.columns=['Neighborhood','Borough',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return df

In [18]:
venue=get_top_places(new_df['Neighborhood'],new_df['Borough'],new_df['Latitude'],new_df['Longitude'])


In [19]:
venue.shape

(3042, 8)

In [20]:
col=['Neighborhood','Borough','Venue Category']

venue.head()

,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,Downtown Toronto,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,Downtown Toronto,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
2,Rosedale,Downtown Toronto,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
3,Rosedale,Downtown Toronto,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
4,Rosedale,Downtown Toronto,43.679563,-79.377529,Pie Squared,43.672143,-79.377856,Pie Shop


In [21]:
venue.groupby(['Neighborhood']).count().shape

(37, 7)

## Grouping the data and creating dummies

In [26]:
venues2=pd.get_dummies(venue[['Venue Category']],prefix="",prefix_sep="")

In [33]:
venues2=pd.concat([venue['Neighborhood'],venues2],axis=1)

In [31]:
venues2.drop('Neighborhood',axis=1,inplace=True)

In [34]:
venue_group=venues2.groupby('Neighborhood').mean().reset_index()

In [35]:
venue_group.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.01,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,...,0.02,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
2,"CN Tower , King and Spadina , Railway Lands , ...",0.00,0.0,0.071429,0.071429,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
3,Central Bay Street,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,...,0.02,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.01,0.0
4,Christie,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,...,0.02,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.00,0.0


## Finding most common Venues

In [43]:
def return_most(row,le):
    r=row.iloc[1:].sort_values(ascending=False)
    return r.index.values[0:le]

In [46]:
columns=['Neighborhood']
for i in range(10):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
venue_most=pd.DataFrame(columns=columns)
venue_most['Neighborhood']=venue_group['Neighborhood']
for i in range(venue_group.shape[0]):
    venue_most.iloc[i,1:]=return_most(venue_group.iloc[i,:],10)
venue_most.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Beer Bar,Bakery,Cocktail Bar,BBQ Joint,Creperie
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Arts & Crafts Store,Park
2,"CN Tower , King and Spadina , Railway Lands , ...",Harbor / Marina,Coffee Shop,Scenic Lookout,Airport,Airport Lounge,Dog Run,Garden,Sculpture Garden,Track,Dance Studio
3,Central Bay Street,Coffee Shop,Japanese Restaurant,Italian Restaurant,Art Gallery,Park,Gastropub,Chinese Restaurant,Tea Room,Ramen Restaurant,Thai Restaurant
4,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Cocktail Bar,Mexican Restaurant,Ethiopian Restaurant,Karaoke Bar,Japanese Restaurant


## Clustering

In [56]:
from sklearn.cluster import KMeans
import matplotlib
!pip install folium
import folium

     |████████████████████████████████| 92kB 9.3MB/s eta 0:00:011


In [52]:
venue_cluster=venue_group.drop('Neighborhood',axis=1)
km=KMeans(init="k-means++",n_clusters=5,n_init=10).fit(venue_cluster)
labels=km.labels_

In [74]:
venue_group.insert(1,'Cluster',labels)
venue_group.head()

,Neighborhood,Cluster,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,1,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,...,0.01,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
1,"Brockton , Parkdale Village , Exhibition Place",2,0.01,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,...,0.02,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
2,"CN Tower , King and Spadina , Railway Lands , ...",0,0.00,0.0,0.071429,0.071429,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0
3,Central Bay Street,1,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,...,0.02,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.01,0.0
4,Christie,2,0.00,0.0,0.000000,0.000000,0.01,0.0,0.0,0.0,...,0.02,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.00,0.0


In [60]:
new_df.sort_values('Neighborhood',inplace=True)
new_df.insert(1,'Cluster',labels)
new_df.head()

,Postal Code,Cluster,Borough,Neighborhood,Latitude,Longitude
6,M5E,1,Downtown Toronto,Berczy Park,43.644771,-79.373306
23,M6K,2,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
14,M5V,0,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420
7,M5G,1,Downtown Toronto,Central Bay Street,43.657952,-79.387383
16,M6G,2,Downtown Toronto,Christie,43.669542,-79.422564


In [57]:
matplotlib.colors.cnames

{'aliceblue': '#F0F8FF',
 'antiquewhite': '#FAEBD7',
 'aqua': '#00FFFF',
 'aquamarine': '#7FFFD4',
 'azure': '#F0FFFF',
 'beige': '#F5F5DC',
 'bisque': '#FFE4C4',
 'black': '#000000',
 'blanchedalmond': '#FFEBCD',
 'blue': '#0000FF',
 'blueviolet': '#8A2BE2',
 'brown': '#A52A2A',
 'burlywood': '#DEB887',
 'cadetblue': '#5F9EA0',
 'chartreuse': '#7FFF00',
 'chocolate': '#D2691E',
 'coral': '#FF7F50',
 'cornflowerblue': '#6495ED',
 'cornsilk': '#FFF8DC',
 'crimson': '#DC143C',
 'cyan': '#00FFFF',
 'darkblue': '#00008B',
 'darkcyan': '#008B8B',
 'darkgoldenrod': '#B8860B',
 'darkgray': '#A9A9A9',
 'darkgreen': '#006400',
 'darkgrey': '#A9A9A9',
 'darkkhaki': '#BDB76B',
 'darkmagenta': '#8B008B',
 'darkolivegreen': '#556B2F',
 'darkorange': '#FF8C00',
 'darkorchid': '#9932CC',
 'darkred': '#8B0000',
 'darksalmon': '#E9967A',
 'darkseagreen': '#8FBC8F',
 'darkslateblue': '#483D8B',
 'darkslategray': '#2F4F4F',
 'darkslategrey': '#2F4F4F',
 'darkturquoise': '#00CED1',
 'darkviolet': '#9400D3

## Visualizing in Map

In [72]:
color=['#00008B','#556B2F',"#FF8C00",'#CD5C5C','#48D1CC']
tor_map=folium.Map(location=[43.6532,-79.3832],zoom_start=12)
for name,lat,lon,cl in zip(new_df['Neighborhood'],new_df['Latitude'],new_df["Longitude"],new_df["Cluster"]):
    folium.CircleMarker(location=[lat,lon],
                                 popup=name,
                                 color=color[cl],
                                 radius=5,
                                 fill_color=color[cl],
                                 fill_opacity=0.6
    ).add_to(tor_map)
tor_map

In [77]:
venue_most.insert(1,'Cluster',labels)

## Examining Clusters

In [78]:
venue_most[venue_most['Cluster']==0]

,Neighborhood,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"CN Tower , King and Spadina , Railway Lands , ...",0,Harbor / Marina,Coffee Shop,Scenic Lookout,Airport,Airport Lounge,Dog Run,Garden,Sculpture Garden,Track,Dance Studio


In [79]:
venue_most[venue_most['Cluster']==1]

,Neighborhood,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,1,Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Beer Bar,Bakery,Cocktail Bar,BBQ Joint,Creperie
3,Central Bay Street,1,Coffee Shop,Japanese Restaurant,Italian Restaurant,Art Gallery,Park,Gastropub,Chinese Restaurant,Tea Room,Ramen Restaurant,Thai Restaurant
5,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Park,Gay Bar,Restaurant,Diner,Café,Italian Restaurant,Bookstore,Pizza Place
6,"Commerce Court , Victoria Hotel",1,Coffee Shop,Hotel,Restaurant,Café,Japanese Restaurant,Concert Hall,Beer Bar,Seafood Restaurant,Bar,Italian Restaurant
10,"First Canadian Place , Underground city",1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Theater,Italian Restaurant,Asian Restaurant,Beer Bar,Deli / Bodega
12,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Italian Restaurant,Electronics Store,Fast Food Restaurant,Department Store
13,"Harbourfront East , Union Station , Toronto Is...",1,Coffee Shop,Hotel,Café,Restaurant,Park,Brewery,Japanese Restaurant,Italian Restaurant,Baseball Stadium,Theater
23,"Richmond , Adelaide , King",1,Coffee Shop,Hotel,Restaurant,Café,Theater,Pizza Place,Bakery,Furniture / Home Store,Movie Theater,Seafood Restaurant
27,St. James Town,1,Coffee Shop,Restaurant,Café,Seafood Restaurant,Italian Restaurant,Hotel,Bakery,Breakfast Spot,Cosmetics Shop,BBQ Joint
28,"St. James Town , Cabbagetown",1,Park,Japanese Restaurant,Coffee Shop,Diner,Café,Restaurant,Gastropub,Thai Restaurant,Pub,Performing Arts Venue


In [80]:
venue_most[venue_most['Cluster']==2]

,Neighborhood,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Brockton , Parkdale Village , Exhibition Place",2,Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Arts & Crafts Store,Park
4,Christie,2,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Cocktail Bar,Mexican Restaurant,Ethiopian Restaurant,Karaoke Bar,Japanese Restaurant
9,"Dufferin , Dovercourt Village",2,Café,Park,Coffee Shop,Bar,Sushi Restaurant,Portuguese Restaurant,Bakery,Pharmacy,Gourmet Shop,Brewery
14,"High Park , The Junction South",2,Café,Bar,Coffee Shop,Convenience Store,Italian Restaurant,Thai Restaurant,Park,Sushi Restaurant,Pizza Place,Mexican Restaurant
15,"India Bazaar , The Beaches West",2,Indian Restaurant,Coffee Shop,Café,Beach,Harbor / Marina,Park,Fast Food Restaurant,Brewery,Burrito Place,Bakery
16,"Kensington Market , Chinatown , Grange Park",2,Café,Bar,Vegetarian / Vegan Restaurant,Bakery,Vietnamese Restaurant,Art Gallery,Coffee Shop,Mexican Restaurant,Dessert Shop,Farmers Market
18,"Little Portugal , Trinity",2,Bar,Café,Restaurant,Coffee Shop,Bakery,Pizza Place,Cocktail Bar,Wine Bar,Italian Restaurant,Asian Restaurant
21,"Parkdale , Roncesvalles",2,Café,Sushi Restaurant,Pizza Place,Coffee Shop,Bar,Restaurant,Pub,Breakfast Spot,Grocery Store,Thai Restaurant
26,"Runnymede , Swansea",2,Coffee Shop,Café,Pizza Place,Bakery,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Gastropub,Pub,Spa
29,Studio District,2,Coffee Shop,Bar,Café,American Restaurant,Bakery,Diner,Brewery,Vietnamese Restaurant,Italian Restaurant,French Restaurant


In [81]:
venue_most[venue_most['Cluster']==3]

,Neighborhood,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,3,Café,Park,Bus Line,Gym / Fitness Center,Bookstore,College Quad,College Gym,Coffee Shop,Trail,Zoo


In [82]:
venue_most[venue_most['Cluster']==4]

,Neighborhood,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Davisville,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Dessert Shop,Pub,Restaurant,Pizza Place,Middle Eastern Restaurant,Gym
8,Davisville North,4,Coffee Shop,Italian Restaurant,Dessert Shop,Café,Gym,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Pharmacy,Diner
11,Forest Hill North & West,4,Coffee Shop,Bank,Café,Park,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Trail,Burger Joint,Pharmacy
19,"Moore Park , Summerhill East",4,Coffee Shop,Grocery Store,Italian Restaurant,Thai Restaurant,Gym,Park,Restaurant,Playground,Pub,Sandwich Place
20,North Toronto West,4,Coffee Shop,Italian Restaurant,Mexican Restaurant,Skating Rink,Diner,Café,Park,Food & Drink Shop,Sushi Restaurant,Jazz Club
22,"Regent Park , Harbourfront",4,Coffee Shop,Pub,Diner,Park,Café,Theater,Italian Restaurant,Restaurant,Bakery,Sushi Restaurant
24,Rosedale,4,Coffee Shop,Grocery Store,Park,Filipino Restaurant,BBQ Joint,Convenience Store,Playground,Office,Sandwich Place,Candy Store
25,Roselawn,4,Sushi Restaurant,Italian Restaurant,Coffee Shop,Pharmacy,Bank,Café,Japanese Restaurant,Bakery,Bagel Shop,Lingerie Store
30,"Summerhill West , Rathnelly , South Hill , For...",4,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Restaurant,Grocery Store,Café,Spa,French Restaurant
